# Testing Non-Linear Products Valuation

This notebook walks through:

1. Loading historical index fixings  
2. Building a simple two-index yield curve  
3. Building **classic** SABR surfaces (no product split)  
4. Pricing IBOR caplets/floors and swaptions using classic Hagan SABR  
5. Building **top-down** SABR surfaces for SOFR (CAPLET vs SWAPTION)  
6. Pricing RFR caplets/swaptions using top-down SABR  


# 1. Imports & Setup

In [16]:
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

import pandas as pd
from yield_curve import YieldCurve
from sabr import SabrModel
from valuation import IndexManager, ValuationEngineRegistry
from product import (
    ProductIborCapFloorlet, ProductOvernightCapFloorlet,
    ProductIborCapFloor,   ProductOvernightCapFloor,
    ProductIborSwaption,   ProductOvernightSwaption
)
from analytics import SABRCalculator

print("Setup complete.")

Setup complete.


## 2) Load Index Fixings

On first call, `IndexManager.instance()` will read **`fixing/fixings.csv`** and populate its in-memory registry.

In [17]:
mgr = IndexManager.instance()
print("Loaded indices:", list(mgr._fixings.keys()))


Loaded indices: ['SOFR-1B', 'USD-LIBOR-3M', 'SONIA-1B']


## 3) Build Dummy Yield Curve

We use two indices (SOFR-1B and USD-LIBOR-BBA-1M) with piecewise-constant interpolation.


In [18]:
curve_data = [
    ["SOFR-1B",          "1M", 0.0020],
    ["SOFR-1B",          "3M", 0.0025],
    ["USD-LIBOR-BBA-1M", "1M", 0.0030],
    ["USD-LIBOR-BBA-1M", "3M", 0.0035]
]
df_curve = pd.DataFrame(curve_data, columns=["INDEX","AXIS1","VALUES"])

build_methods_curve = [
    {"TARGET":"SOFR-1B",           "INTERPOLATION METHOD":"PIECEWISE_CONSTANT"},
    {"TARGET":"USD-LIBOR-BBA-1M",  "INTERPOLATION METHOD":"PIECEWISE_CONSTANT"}
]

valueDate = "2025-06-26"

yc = YieldCurve(valueDate, df_curve, build_methods_curve)
print("YieldCurve built. Components:", yc.components.keys())


YieldCurve built. Components: dict_keys(['SOFR-1B', 'USD-LIBOR-BBA-1M'])


## 4) Build Dummy SABR Model

We’ll calibrate a flat SABR surface for USD-LIBOR-BBA-1M so that our caplets can be priced.

In [19]:
sabr_libor = pd.DataFrame([
    ["USD-LIBOR-BBA-1M", 0.25, 0.25, 0.015, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 0.25, 1.00, 0.017, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 1.00, 0.25, 0.018, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 1.00, 1.00, 0.020, 0.5, 0.2, -0.3],
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

sabr_sofr = pd.DataFrame([
    ["SOFR-1B", 0.25, 0.25, 0.010, 0.5, 0.2, -0.1],
    ["SOFR-1B", 0.25, 1.00, 0.012, 0.5, 0.2, -0.1],
    ["SOFR-1B", 1.00, 0.25, 0.013, 0.5, 0.2, -0.1],
    ["SOFR-1B", 1.00, 1.00, 0.015, 0.5, 0.2, -0.1],
], columns=sabr_libor.columns)

sabr_data = pd.concat([sabr_libor, sabr_sofr], ignore_index=True)

classic_build_methods = []
for idx in ("USD-LIBOR-BBA-1M", "SOFR-1B"):
    for param in ("NORMALVOL","BETA","NU","RHO"):
        classic_build_methods.append({
            "TARGET":        idx,
            "AXIS1":         "AXIS1",
            "AXIS2":         "AXIS2",
            "VALUES":        param,
            "INTERPOLATION": "LINEAR",
            "SHIFT":         0.0,
            "VOL_DECAY_SPEED": 0.2
        })

classic_sabr = SabrModel.from_data(
    valueDate,
    sabr_data,
    classic_build_methods,
    df_curve, build_methods_curve
)
print("Classic SABR components:", classic_sabr.components.keys())

Classic SABR components: dict_keys(['USD-LIBOR-BBA-1M-NORMALVOL', 'USD-LIBOR-BBA-1M-BETA', 'USD-LIBOR-BBA-1M-NU', 'USD-LIBOR-BBA-1M-RHO', 'SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


## 5) Price with Classic SABR

In [20]:
caplet_ibor = ProductIborCapFloorlet(
    startDate   ="2025-07-01",
    endDate     ="2025-10-01",
    index       ="USD-LIBOR-BBA-1M",
    optionType  ="CAP",
    strike      =0.02,
    notional    =1_000_000,
    longOrShort ="LONG"
)

ve1 = ValuationEngineRegistry().new_valuation_engine(
    classic_sabr,
    {"SABR_METHOD": None},   # None → classic Hagan
    caplet_ibor
)
ve1.calculateValue()
print("IBOR Caplet PV (classic):", ve1.value)

IBOR Caplet PV (classic): ['USD', np.float64(4.688928821898315e-06)]


In [21]:
swaption_ibor = ProductIborSwaption(
    optionExpiry="2025-12-01",
    swapStart   ="2026-01-01",
    swapEnd     ="2031-01-01",
    frequency   ="3M",
    iborIndex   ="USD-LIBOR-BBA-1M",
    strikeRate  =0.0175,
    notional    =1_000_000,
    longOrShort ="LONG",
    optionType  ="PAYER"
)

ve2 = ValuationEngineRegistry().new_valuation_engine(
    classic_sabr,
    {"SABR_METHOD": None},
    swaption_ibor
)
ve2.calculateValue()
print("IBOR Swaption PV (classic):", ve2.value)

IBOR Swaption PV (classic): ['USD', np.float64(17064241489.76993)]


In [22]:
swaption_ibor = ProductIborSwaption(
    optionExpiry="2025-12-01",
    swapStart   ="2026-01-01",
    swapEnd     ="2031-01-01",
    frequency   ="3M",
    iborIndex   ="USD-LIBOR-BBA-1M",
    strikeRate  =0.0175,
    notional    =1000000,
    longOrShort ="LONG",
    optionType  ="PAYER"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    classic_sabr,
    {"SABR_METHOD": None},
    swaption_ibor
)
ve.calculateValue()
print("IBOR Swaption PV:", ve.value)

IBOR Swaption PV: ['USD', np.float64(17064241489.76993)]


## 6) Build Top-Down SABR Model for SOFR (CAPLET vs SWAPTION)


In [23]:
sabr_sofr_cap = pd.DataFrame([
    ["SOFR-1B", 0.25, 0.083333, 0.0090,    0.5,  0.20, -0.15],
    ["SOFR-1B", 0.25, 0.25,     0.0100,    0.5,  0.20, -0.15],
    ["SOFR-1B", 1.00, 0.083333, 0.0110,    0.5,  0.20, -0.15],
    ["SOFR-1B", 1.00, 0.25,     0.0120,    0.5,  0.20, -0.15],
], columns=sabr_sofr.columns)
sabr_sofr_cap["PRODUCT"] = "CAPLET"

sabr_sofr_sw = pd.DataFrame([
    ["SOFR-1B",0.25,1.0,0.012,0.5,0.2,-0.1],
    ["SOFR-1B",0.25,2.0,0.013,0.5,0.2,-0.1],
    ["SOFR-1B",0.25,3.0,0.014,0.5,0.2,-0.1],
    ["SOFR-1B",0.25,4.0,0.015,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,1.0,0.013,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,2.0,0.014,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,3.0,0.015,0.5,0.2,-0.1],
    ["SOFR-1B",1.00,4.0,0.016,0.5,0.2,-0.1]
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

sabr_sofr_sw["PRODUCT"] = "SWAPTION"

full_sabr_sofr = pd.concat([sabr_sofr_cap, sabr_sofr_sw], ignore_index=True)

topdown_build_methods = []

for param in ("NORMALVOL","BETA","NU","RHO"):
    topdown_build_methods.append({
        "TARGET":        "SOFR-1B",
        "AXIS1":         "AXIS1",
        "AXIS2":         "AXIS2",
        "VALUES":        param,
        "INTERPOLATION": "LINEAR",
        "SHIFT":         0.0,
        "VOL_DECAY_SPEED":0.2,
        "PRODUCT":       "CAPLET"
    })

for param in ("NORMALVOL","BETA","NU","RHO"):
    topdown_build_methods.append({
        "TARGET":        "SOFR-1B",
        "AXIS1":         "AXIS1",
        "AXIS2":         "AXIS2",
        "VALUES":        param,
        "INTERPOLATION": "LINEAR",
        "SHIFT":         0.0,
        "VOL_DECAY_SPEED":0.2,
        "PRODUCT":       "SWAPTION"
    })

td_sabr = SabrModel.from_data(
    valueDate,
    full_sabr_sofr,
    topdown_build_methods,
    df_curve, build_methods_curve
)
print("Top-down SABR components:", td_sabr.components.keys())


Top-down SABR components: dict_keys(['SOFR-1B-NORMALVOL-CAPLET', 'SOFR-1B-BETA-CAPLET', 'SOFR-1B-NU-CAPLET', 'SOFR-1B-RHO-CAPLET', 'SOFR-1B-NORMALVOL-SWAPTION', 'SOFR-1B-BETA-SWAPTION', 'SOFR-1B-NU-SWAPTION', 'SOFR-1B-RHO-SWAPTION'])


## 7) Price Top-Down RFR Products


In [24]:
# 7a) Overnight caplet (top-down SABR)
caplet_ois = ProductOvernightCapFloorlet(
    effectiveDate="2025-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1_000_000,
    longOrShort  ="LONG"
)

ve3 = ValuationEngineRegistry().new_valuation_engine(
    td_sabr,
    {"SABR_METHOD":"top-down"},
    caplet_ois
)
ve3.calculateValue()
print("Overnight Caplet PV (top-down):", ve3.value)

Overnight Caplet PV (top-down): ['USD', np.float64(70.68822914416128)]


In [25]:
# 7b) Overnight swaption (top-down SABR)
swaption_ois = ProductOvernightSwaption(
    optionExpiry  ="2025-10-01",
    swapStart     ="2025-11-01",
    swapEnd       ="2027-11-01",
    frequency     ="1M",
    overnightIndex="SOFR-1B",
    strikeRate    =0.01,
    notional      =500_000,
    longOrShort   ="SHORT",
    optionType    ="PAYER"
)

ve4 = ValuationEngineRegistry().new_valuation_engine(
    td_sabr,
    {"SABR_METHOD":"top-down"},
    swaption_ois
)
ve4.calculateValue()
print("Overnight Swaption PV (top-down):", ve4.value)

Overnight Swaption PV (top-down): ['USD', np.float64(20316412353.452965)]


## 8) Valuation: Overnight Cap/Floor Wrapper


In [26]:
ois_cap = ProductOvernightCapFloor(
    effectiveDate="2025-07-01",
    maturityDate ="2026-07-01",
    frequency    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="FLOOR",
    strike       =0.016,
    notional     =1000000,
    longOrShort  ="SHORT"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    td_sabr,
    {"SABR_METHOD": "top-down"},
    ois_cap
)
ve.calculateValue()
print("Overnight Cap/Floor PV:", ve.value)


Overnight Cap/Floor PV: ['USD', np.float64(-15435.102213152948)]


# 8) Price RFR caplets using bottom-up SABR

In [27]:
# build a toy correlation surface: EXPIRY x TENOR → CORR
corr_data = [
    [0.25, 0.25, 0.80],
    [0.25, 0.50, 0.78],
    [0.25, 1.00, 0.75],
    [1.00, 0.25, 0.78],
    [1.00, 0.50, 0.76],
    [1.00, 1.00, 0.74],
]
df_corr = pd.DataFrame(corr_data, columns=["EXPIRY","TENOR","CORR"])

In [28]:
one_bd = 1.0/252

sabr_sofr_1bd = pd.DataFrame([
    ["SOFR-1B", 0.25,  one_bd,  0.0095,    0.50, 0.20, -0.15],
    ["SOFR-1B", 0.50,  one_bd,  0.0100,    0.50, 0.20, -0.15],
    ["SOFR-1B", 1.00,  one_bd,  0.0105,    0.50, 0.20, -0.15],
    ["SOFR-1B", 2.00,  one_bd,  0.0110,    0.50, 0.20, -0.15]
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

In [29]:
bottom_up_build_methods_sofr = [
    {
      "TARGET":   "SOFR-1B",
      "AXIS1":    "AXIS1",
      "AXIS2":    "AXIS2",
      "VALUES":   p,
      "INTERPOLATION":"LINEAR",
      "SHIFT":    0.0,
      "VOL_DECAY_SPEED": 0.2
    }
    for p in ("NORMALVOL","BETA","NU","RHO")
]

bottom_up_sofr_sabr = SabrModel.from_data(
    valueDate,
    sabr_sofr_1bd,
    bottom_up_build_methods_sofr,
    df_curve,
    build_methods_curve
)
print("Bottom-up 1BD‐tenor SABR components:", bottom_up_sofr_sabr.components.keys())



Bottom-up 1BD‐tenor SABR components: dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [30]:
caplet_bu = ProductOvernightCapFloorlet(
    effectiveDate="2025-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1_000_000,
    longOrShort  ="LONG"
)
ve_bu_cap = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr,
    {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr},
    caplet_bu
)
ve_bu_cap.calculateValue()
print("Overnight Caplet PV (bottom-up):", ve_bu_cap.value)


Overnight Caplet PV (bottom-up): ['USD', np.float64(3.142194825426369e-32)]


In [36]:
ois_cap = ProductOvernightCapFloor(
    effectiveDate="2025-07-01",
    maturityDate ="2026-07-01",
    frequency    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1_000_000,
    longOrShort  ="LONG"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    bottom_up_sofr_sabr,
    {"SABR_METHOD":"bottom-up", "CORR_DF": df_corr},
    ois_cap
)
ve.calculateValue()
print("Overnight Cap PV (bottom-up):", ve.value)

Overnight Cap PV (bottom-up): ['USD', np.float64(2.0843399280856785)]
